# DATA PREPROCESSING

In this code we will fetch the missing datas about the stock like importing the Sectors and Industry of the stock they are trading, buy and sell time and date. Preprocess the data, perform Data Cleaning, perform Exploratory Data Analysis and at last will do Feature Engineering to find the hidden patterns and trends in the data.

Now we will import the necessary libaries.

In [ ]:
import pandas as pd
import numpy as np  
import yfinance as yf
from datetime import datetime  
import matplotlib.pyplot as plt
import seaborn as sns

Now we will load the Dataset to Preprocess.

In [ ]:
import pandas as pd

def load_clean_excel(file_path, start_keywords=['Stock name'], end_keywords=['Unrealised trades', 'Disclaimer']):
    # Preview file to find header and footer
    preview = pd.read_excel(file_path, header=None)
    
    # Find where the actual table starts
    start_mask = preview.apply(lambda row: row.astype(str).str.contains('|'.join(start_keywords), case=False, na=False)).any(axis=1)
    start_index = start_mask[start_mask].index.min()
    
    # Find where table ends
    end_mask = preview.apply(lambda row: row.astype(str).str.contains('|'.join(end_keywords), case=False, na=False)).any(axis=1)
    end_index = end_mask[end_mask].index.min() if end_mask.any() else len(preview)
    
    print(f"{file_path}")
    print(f"Data starts at row {start_index}, ends before row {end_index}")
    
    # Read only the relevant data rows
    df = pd.read_excel(file_path, header=start_index, nrows=end_index - start_index - 1)
    
    # Clean column names
    df.columns = (df.columns
                  .astype(str)
                  .str.strip()
                  .str.replace('\u00A0', ' ', regex=True)
                  .str.replace('\s+', ' ', regex=True))
    
    # Drop empty rows/columns
    df.dropna(how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    
    print(f" Clean Data Loaded — Shape: {df.shape}\n")
    return df


# Step 1: File Paths
pnl_path = "Stocks_PnL_Report_3788142010_01-10-2020_06-11-2025.xlsx"
order_path = "Stocks_Order_History_3788142010_01-10-2020_06-11-2025.xlsx"

# Step 2: Load both files dynamically
pnl = load_clean_excel(pnl_path)
order_history = load_clean_excel(order_path)

# Step 3: Verify loaded datasets
print("PnL Columns:", pnl.columns.tolist())
print("Order History Columns:", order_history.columns.tolist())

# Optional — display samples
print("\nPnL Preview:")
print(pnl.head(3))

print("\nOrder History Preview:")
print(order_history.head(3))


Now let's find some information about the data

In [ ]:
pnl.info()
pnl.shape
pnl.describe()

similarly for the Order's history

In [ ]:
order_history.info()
order_history.shape
order_history.describe()

Now Let's see how the data looks 

In [ ]:
pnl.head()
pnl.sample(5)

Similarly for the Order history

In [ ]:
order_history.head()
order_history.sample(5)

Let's check for the missing values and the duplicated values

In [ ]:
pnl.isnull().sum()

In [ ]:
pnl.duplicated().sum()

Similarly for the order history

In [ ]:
order_history.isnull().sum()

In [ ]:
order_history.duplicated().sum()

# Data Fetching

Now lets fetch the execution date and time for both the buy and sell orders from the order history to the pnl to do the further preprocesses and find the time-based analysis. 

In [ ]:
# Ensure datetime column exists and is clean
order_history.columns = order_history.columns.str.strip().str.replace('\u00A0', ' ', regex=True)
order_history['Execution date and time'] = pd.to_datetime(order_history['Execution date and time'], errors='coerce')

# Split date and time safely
order_history['Execution Date'] = order_history['Execution date and time'].dt.date
order_history['Execution Time'] = order_history['Execution date and time'].dt.time

# ---------- Step 3: Clean and convert buy/sell dates safely ----------
pnl['Buy date'] = pd.to_datetime(pnl['Buy date'], errors='coerce').dt.date
pnl['Sell date'] = pd.to_datetime(pnl['Sell date'], errors='coerce').dt.date

# Drop any rows where dates failed to parse
pnl = pnl.dropna(subset=['Buy date', 'Sell date'])

# ---------- Step 4: Define market hours helper ----------
def is_market_time(dt):
    if pd.notna(dt):
        time = dt.time()
        return (time >= pd.Timestamp('09:15:00').time()) and (time <= pd.Timestamp('15:30:00').time())
    return False

# ---------- Step 5: Function to get times from order history ----------
def get_times_from_order_history(pnl_row, order_df):
    stock_name = pnl_row['Stock name']
    isin = pnl_row['ISIN']
    buy_date = pnl_row['Buy date']
    sell_date = pnl_row['Sell date']
    
    stock_orders = order_df[
        (order_df['Stock name'] == stock_name) &
        (order_df['ISIN'] == isin)
    ].copy()
    
    # Case 1: Intraday (same date)
    if buy_date == sell_date:
        buy_order = stock_orders[
            (stock_orders['Execution date and time'].dt.date == buy_date) &
            (stock_orders['Type'].str.upper() == 'BUY') &
            (stock_orders['Execution date and time'].apply(is_market_time))
        ].sort_values('Execution date and time').head(1)
        
        sell_order = stock_orders[
            (stock_orders['Execution date and time'].dt.date == sell_date) &
            (stock_orders['Type'].str.upper() == 'SELL') &
            (stock_orders['Execution date and time'].apply(is_market_time))
        ].sort_values('Execution date and time').tail(1)
    
    # Case 2: Delivery
    else:
        buy_order = stock_orders[
            (stock_orders['Execution date and time'].dt.date == buy_date) &
            (stock_orders['Type'].str.upper() == 'BUY') &
            (stock_orders['Execution date and time'].apply(is_market_time))
        ].sort_values('Execution date and time').head(1)
        
        sell_order = stock_orders[
            (stock_orders['Execution date and time'].dt.date == sell_date) &
            (stock_orders['Type'].str.upper() == 'SELL') &
            (stock_orders['Execution date and time'].apply(is_market_time))
        ].sort_values('Execution date and time').tail(1)
    
    # Extract times or set defaults
    buy_time = buy_order['Execution date and time'].iloc[0].time() if not buy_order.empty else pd.Timestamp('09:15:00').time()
    sell_time = sell_order['Execution date and time'].iloc[0].time() if not sell_order.empty else pd.Timestamp('15:30:00').time()
    
    return buy_time, sell_time

# ---------- Step 6: Apply function ----------
pnl[['Buy Time', 'Sell Time']] = pnl.apply(
    lambda row: pd.Series(get_times_from_order_history(row, order_history)), axis=1
)

print("Buy/Sell times added successfully!")
print(pnl[['Stock name', 'Buy date', 'Buy Time', 'Sell date', 'Sell Time']].head(20))


Similarly, now let's fetch the sectors and industry for each of the trades to find the sector-based and industry-based trends and patterns in the user trade history 

In [ ]:
# Create symbol mapping from order history (Symbol to Stock name)
symbol_to_name = dict(zip(order_history['Symbol'], order_history['Stock name']))
name_to_symbol = {v: k for k, v in symbol_to_name.items()}

# Get unique symbols from order history
unique_symbols = order_history['Symbol'].unique()

# Fetch sector data
sector_data = {}
for symbol in unique_symbols:
    try:
        ticker = yf.Ticker(f"{symbol}.NS")
        info = ticker.info
        sector_data[symbol] = (
            info.get('sector', 'Unknown'),
            info.get('industry', 'Unknown')
        )
    except Exception as e:
        print(f"Error fetching {symbol}: {str(e)}")
        sector_data[symbol] = ('Unknown', 'Unknown')

# Add to order history
order_history['Sector'] = order_history['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[0])
order_history['Industry'] = order_history['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[1])

# Add to PnL - first extract symbol from stock name
pnl['Symbol'] = pnl['Stock name'].map(name_to_symbol)
pnl['Sector'] = pnl['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[0])
pnl['Industry'] = pnl['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[1])

print("Sector and Industry data added successfully!")

Since we imported the necessary Timing and Sectorial datas for the trades, now we will be moving towards the Data Cleaning part and afterwards we will apply some EDA techniques to find some hidden patterns.  

# Data Cleaning

In [ ]:
#Let's check the information of the datasets again
pnl.info()
pnl.shape

Since Remarks have so many missing values, lets fill them

In [ ]:
pnl['Remark'].value_counts()

In [ ]:
pnl['Remark'] = pnl['Remark'].fillna('Delivery Trade')
pnl['Remark'].value_counts()

Since we have duplicate values, let's deal with them

In [ ]:
pnl = pnl.drop_duplicates()
pnl.duplicated().sum()
pnl.shape

Let's handle the missing Sector and Industry names.

In [ ]:
pnl['Sector'].to_list()
pnl['Sector'] = pnl['Sector'].replace(['Unknown', ''], 'Others')

In [ ]:
pnl['Industry'].to_list()
pnl['Industry'] = pnl['Industry'].replace(['Unknown', ''], 'Others')

# Exploratory Data Analysis

In [ ]:
pnl.info()
pnl.describe()

1. Univariate Analysis

Let's do the Univariate Analysis for the numerical features

In [ ]:
# Select numeric columns
numerical_cols = pnl.select_dtypes(include=['int64', 'float64']).columns

# Dynamically set rows and columns for subplot grid
n_cols = 3
n_rows = (len(numerical_cols) + n_cols - 1) // n_cols

# Create figure
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
axes = axes.flatten()

# Plot each numeric column
for i, col in enumerate(numerical_cols):
    sns.histplot(pnl[col].dropna(), kde=True, ax=axes[i], bins=30, color='skyblue')
    axes[i].set_title(f"Distribution of {col}", fontsize=12, weight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Frequency")
    axes[i].grid(alpha=0.3)

# Remove any unused axes (in case of fewer numeric columns)
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


These plots tells us that there are too much skewness in the numerical cols. Also explaining that there are too much correlation between the Numerical cols

Now, let's do the Univariate Analysis for the Categorical features

In [ ]:
# Merging the Industry rows appearing less than 4 times
uncommon_industry = pnl['Industry'].value_counts() 
pnl['Industry'] = pnl['Industry'].replace(uncommon_industry[uncommon_industry < 4].index, 'Others')

In [ ]:
# Merging the Symbols rows appearing less than 5 times
uncommon_Symbols = pnl['Symbol'].value_counts() 
pnl['Symbol'] = pnl['Symbol'].replace(uncommon_Symbols[uncommon_Symbols < 4].index, 'Others')

In [ ]:
categorical_cols = pnl[['Remark', 'Symbol', 'Sector', 'Industry']]

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(18, 15))
axes = axes.flatten()  # Flatten for easy iteration

# Plot each categorical column
for i, col in enumerate(categorical_cols):
    value_counts = pnl[col].value_counts().head(20)  # top 15 categories
    sns.barplot(x=value_counts.index, y=value_counts.values, ax=axes[i], palette='pastel')
    axes[i].set_title(f"Count Plot of {col}", fontsize=12)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Count")
    axes[i].tick_params(axis='x', rotation=60)

# Remove empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

First plot tells us that the user has traded mostly intraday then swing trades, also got lucky to get some IPO's allotment and got bonus shares for some of the holdings he had. 
Second plot tells us that user had traded different stocks while he had traded IndusInd Bank stock the most number of times. Third and Fourth plot tells us that the user had traded the stocks which comes from the different types of sectors and industry but his most traded were Financial Services and Regional Banks.

Let's plot the graph for the rest of the categorical features

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(19, 10))
sns.set(style='whitegrid')

# Ensure datetime types
pnl['Buy date'] = pd.to_datetime(pnl['Buy date'], errors='coerce')
pnl['Sell date'] = pd.to_datetime(pnl['Sell date'], errors='coerce')

# Format months as 'MMM-YYYY' (e.g., Jan-2025)
pnl['Buy Month'] = pnl['Buy date'].dt.strftime('%b-%Y')
pnl['Sell Month'] = pnl['Sell date'].dt.strftime('%b-%Y')

# Define 1-hour interval bins for market hours (9:00–15:30)
bins = [9, 10, 11, 12, 13, 14, 15, 15.5]
labels = ['09:00–10:00', '10:00–11:00', '11:00–12:00', '12:00–13:00', '13:00–14:00', '14:00–15:00', '15:00–15:30']

# Extract and bin times
pnl['Buy Hour'] = pnl['Buy Time'].apply(lambda x: x.hour + x.minute / 60 if pd.notnull(x) else None)
pnl['Sell Hour'] = pnl['Sell Time'].apply(lambda x: x.hour + x.minute / 60 if pd.notnull(x) else None)
pnl['Buy Interval'] = pd.cut(pnl['Buy Hour'], bins=bins, labels=labels, right=False)
pnl['Sell Interval'] = pd.cut(pnl['Sell Hour'], bins=bins, labels=labels, right=False)

# Buy Time 
axes[0,0].set_title("Distribution of Buy Times (Hourly Intervals)", fontsize=13)
sns.barplot(x=pnl['Buy Interval'].value_counts().reindex(labels).index,
            y=pnl['Buy Interval'].value_counts().reindex(labels).values,
            ax=axes[0,0], color='steelblue')
axes[0,0].set_xlabel("Time Interval")
axes[0,0].set_ylabel("Number of Buys")
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(alpha=0.3)

# Sell Time 
axes[0,1].set_title("Distribution of Sell Times (Hourly Intervals)", fontsize=13)
sns.barplot(x=pnl['Sell Interval'].value_counts().reindex(labels).index,
            y=pnl['Sell Interval'].value_counts().reindex(labels).values,
            ax=axes[0,1], color='indianred')
axes[0,1].set_xlabel("Time Interval")
axes[0,1].set_ylabel("Number of Sells")
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(alpha=0.3)

# Buy Month 
axes[1, 0].set_title("Trades Grouped by Buy Month", fontsize=13)
buy_month_data = pnl['Buy Month'].value_counts().sort_index(key=lambda x: pd.to_datetime(x, format='%b-%Y'))
sns.barplot(x=buy_month_data.index, y=buy_month_data.values, ax=axes[1, 0], color='#1f77b4')
axes[1, 0].set_xlabel("Month-Year")
axes[1, 0].set_ylabel("Number of Buys")
axes[1, 0].tick_params(axis='x', rotation=45)

# Sell Month 
axes[1, 1].set_title("Trades Grouped by Sell Month", fontsize=13)
sell_month_data = pnl['Sell Month'].value_counts().sort_index(key=lambda x: pd.to_datetime(x, format='%b-%Y'))
sns.barplot(x=sell_month_data.index, y=sell_month_data.values, ax=axes[1, 1], color='#d62728')
axes[1, 1].set_xlabel("Month-Year")
axes[1, 1].set_ylabel("Number of Sells")
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

These Plots tells us that the user had buy the most number of the stocks in the first hour of the day and sell stocks in the last hour of the day while also says that the user was trading from the Nov,21 but were mostly active between the June,22 to Sept,22 and Oct,23 to July,24 telling that he is not focused properly or gets unactive after huge losses and also he had taken some gaps between the months were he was completely unactive.

2. Bivariate Analysis

Now, let's do the bivariate analysis. Starting with the plots between the Numerical vs Numerical features

In [ ]:
# Pairplot for all numerical features
sns.pairplot(pnl[numerical_cols], diag_kind='kde', corner=False)
plt.suptitle("Pairwise Relationships between Numerical Variables", y=1.02, fontsize=14)
plt.show()

# Correlation Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(pnl[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap (Numerical Columns)")
plt.show()

---------Plots between Numerical vs Categorical features

In [ ]:
categorical_cols = pnl.select_dtypes(include=['object']).columns
categorical_cols_drop = categorical_cols.drop(['ISIN', 'Stock name'])
target = 'Realised P&L'

if target not in pnl.columns:
    print("Target column 'Realised P&L' not found — skipping plots.")
else:
    for col in categorical_cols_drop:
        n_unique = pnl[col].nunique()
        
        if col in ['Buy Time', 'Sell Time']:
            if col == 'Buy Time':
                fig, axes = plt.subplots(1, 2, figsize=(22, 7))
                
                for idx, time_col in enumerate(['Buy Time', 'Sell Time']):
                    pnl_time = pnl.copy()
                    pnl_time[f'{time_col}_Hour'] = pd.to_datetime(pnl_time[time_col], 
                                                                   format='%H:%M:%S', errors='coerce').dt.hour
                    
                    hourly_data = pnl_time.groupby(f'{time_col}_Hour')[target].agg(['mean', 'count']).reset_index()
                    hourly_data = hourly_data.sort_values(f'{time_col}_Hour')
                    
                    colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in hourly_data['mean']]
                    
                    axes[idx].bar(hourly_data[f'{time_col}_Hour'], hourly_data['mean'], 
                                 color=colors, alpha=0.8, edgecolor='white', linewidth=2)
                    axes[idx].set_title(f"Average {target} by {time_col} (Hourly)", 
                                       fontsize=15, fontweight='bold', pad=15)
                    axes[idx].set_xlabel('Hour of Day', fontsize=12, fontweight='bold')
                    axes[idx].set_ylabel(f'Average {target}', fontsize=12, fontweight='bold')
                    axes[idx].axhline(y=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
                    axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
                    axes[idx].spines['top'].set_visible(False)
                    axes[idx].spines['right'].set_visible(False)
                    axes[idx].set_xticks(hourly_data[f'{time_col}_Hour'])
                    
                    for i, row in hourly_data.iterrows():
                        offset = 20 if row['mean'] > 0 else -20
                        axes[idx].text(row[f'{time_col}_Hour'], row['mean'] + offset, 
                                      f"n={int(row['count'])}", 
                                      ha='center', va='center', fontsize=9, fontweight='bold',
                                      bbox=dict(boxstyle='round,pad=0.4', facecolor='white', 
                                               edgecolor='gray', alpha=0.8))
                
                plt.tight_layout()
                plt.show()
        
        elif col == 'Symbol':
            fig, ax = plt.subplots(figsize=(16, 9))
            stats = pnl.groupby(col)[target].agg(['mean', 'count']).reset_index()
            stats = stats.sort_values('mean', ascending=True)
            
            colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in stats['mean']]
            
            bars = ax.barh(range(len(stats)), stats['mean'], color=colors, alpha=0.8, 
                          edgecolor='white', linewidth=2)
            
            ax.set_yticks(range(len(stats)))
            ax.set_yticklabels(stats[col], fontsize=11, fontweight='bold')
            ax.set_title(f'Average {target} by {col}', fontsize=16, fontweight='bold', pad=20)
            ax.set_xlabel(f'Average {target}', fontsize=13, fontweight='bold')
            ax.set_ylabel(col, fontsize=13, fontweight='bold')
            ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
            ax.grid(axis='x', alpha=0.3, linestyle='--')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            for i, (idx, row) in enumerate(stats.iterrows()):
                offset = 20 if row['mean'] > 0 else -20
                ax.text(row['mean'] + offset, i, f"n={int(row['count'])}", 
                       va='center', ha='center', fontsize=10, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                                edgecolor='gray', alpha=0.8))
            
            plt.tight_layout()
            plt.show()
        
        elif col in ['Sector', 'Industry']:
            fig, axes = plt.subplots(1, 2, figsize=(24, 9))
            
            stats = pnl.groupby(col)[target].agg(['mean', 'sum', 'count']).reset_index()
            stats = stats.sort_values('mean', ascending=True)
            
            colors_mean = ['#2ecc71' if x > 0 else '#e74c3c' for x in stats['mean']]
            bars0 = axes[0].barh(range(len(stats)), stats['mean'], color=colors_mean, 
                                alpha=0.8, edgecolor='white', linewidth=2)
            
            axes[0].set_yticks(range(len(stats)))
            axes[0].set_yticklabels(stats[col], fontsize=11, fontweight='bold', rotation = 30)
            axes[0].set_title(f'Average {target} by {col}', fontsize=14, fontweight='bold', pad=15)
            axes[0].set_xlabel(f'Average {target}', fontsize=12, fontweight='bold')
            axes[0].axvline(x=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
            axes[0].grid(axis='x', alpha=0.3, linestyle='--')
            axes[0].spines['top'].set_visible(False)
            axes[0].spines['right'].set_visible(False)
            
            for i, (idx, row) in enumerate(stats.iterrows()):
                offset = 50 if row['mean'] > 0 else -50
                axes[0].text(row['mean'] + offset, i, f"n={int(row['count'])}", 
                           va='center', ha='center', fontsize=9, fontweight='bold',
                           bbox=dict(boxstyle='round,pad=0.4', facecolor='white', 
                                    edgecolor='gray', alpha=0.8))
            
            colors_total = ['#2ecc71' if x > 0 else '#e74c3c' for x in stats['sum']]
            bars1 = axes[1].barh(range(len(stats)), stats['sum'], color=colors_total, 
                                alpha=0.8, edgecolor='white', linewidth=2)
            
            axes[1].set_yticks(range(len(stats)))
            axes[1].set_yticklabels(stats[col], fontsize=11, fontweight='bold', rotation = 30)
            axes[1].set_title(f'Total {target} by {col}', fontsize=14, fontweight='bold', pad=15)
            axes[1].set_xlabel(f'Total {target}', fontsize=12, fontweight='bold')
            axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
            axes[1].grid(axis='x', alpha=0.3, linestyle='--')
            axes[1].spines['top'].set_visible(False)
            axes[1].spines['right'].set_visible(False)
            
            for i, (idx, row) in enumerate(stats.iterrows()):
                offset = 400 if row['sum'] > 0 else -400
                axes[1].text(row['sum'] + offset, i, f"n={int(row['count'])}", 
                           va='center', ha='center', fontsize=9, fontweight='bold',
                           bbox=dict(boxstyle='round,pad=0.4', facecolor='white', 
                                    edgecolor='gray', alpha=0.8))
            
            plt.tight_layout()
            plt.show()

        elif col == 'Remark':
            fig, ax = plt.subplots(figsize=(8, 7))
            stats = pnl.groupby(col)[target].agg(['mean', 'count']).reset_index()
            stats = stats.sort_values('mean', ascending=False)
            
            colors = ['#2ecc71' if x > 0 else '#e74c3c' for x in stats['mean']]
            
            bars = ax.bar(range(len(stats)), stats['mean'], color=colors, alpha=0.8, 
                         edgecolor='white', linewidth=2)
            
            ax.set_xticks(range(len(stats)))
            ax.set_xticklabels(stats[col], fontsize=11, fontweight='bold', rotation=45, ha='right')
            ax.set_title(f'Average {target} by {col}', fontsize=16, fontweight='bold', pad=20)
            ax.set_ylabel(f'Average {target}', fontsize=13, fontweight='bold')
            ax.set_xlabel(col, fontsize=13, fontweight='bold')
            ax.axhline(y=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            for i, (idx, row) in enumerate(stats.iterrows()):
                offset = 50 if row['mean'] > 0 else -50
                ax.text(i, row['mean'] + offset, f"n={int(row['count'])}", 
                       ha='center', va='center', fontsize=10, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                                edgecolor='gray', alpha=0.8))
            
            plt.tight_layout()
            plt.show()
        
        else:
            fig, ax = plt.subplots(figsize=(16, 7))
            stats = pnl.groupby(col)[target].agg(['mean', 'count']).reset_index()
            stats = stats.sort_values('mean', ascending=True)
            
            colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in stats['mean']]
            
            bars = ax.barh(range(len(stats)), stats['mean'], color=colors, alpha=0.8, 
                          edgecolor='white', linewidth=2)
            
            ax.set_yticks(range(len(stats)))
            ax.set_yticklabels(stats[col], fontsize=11, fontweight='bold')
            ax.set_title(f'Average {target} by {col}', fontsize=16, fontweight='bold', pad=20)
            ax.set_xlabel(f'Average {target}', fontsize=13, fontweight='bold')
            ax.set_ylabel(col, fontsize=13, fontweight='bold')
            ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
            ax.grid(axis='x', alpha=0.3, linestyle='--')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            
            for i, (idx, row) in enumerate(stats.iterrows()):
                offset = 150 if row['mean'] > 0 else -150
                ax.text(row['mean'] + offset, i, f"n={int(row['count'])}", 
                       va='center', ha='center', fontsize=10, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                                edgecolor='gray', alpha=0.8))
            
            plt.tight_layout()
            plt.show()
            

-----------Plots between Categorical vs Categorical features

In [ ]:
important_pairs = [
    ('Buy Month', 'Sell Month'),
    ('Buy Month', 'Sector'),
    ('Sell Month', 'Sector'),
    ('Remark', 'Sector'),
    ('Remark', 'Industry'),
    ('Buy Month', 'Remark'),
    ('Sell Month', 'Remark')
]

month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for col1, col2 in important_pairs:
    if col1 in pnl.columns and col2 in pnl.columns:
        if pnl[col1].nunique() <= 20 and pnl[col2].nunique() <= 20:
            
            if col1 in ['Buy Month', 'Sell Month']:
                pnl_temp = pnl[pnl[col1].isin(month_order)].copy()
                pnl_temp[col1] = pd.Categorical(pnl_temp[col1], categories=month_order, ordered=True)
            else:
                pnl_temp = pnl.copy()
            
            if col2 in ['Buy Month', 'Sell Month']:
                pnl_temp = pnl_temp[pnl_temp[col2].isin(month_order)].copy()
                pnl_temp[col2] = pd.Categorical(pnl_temp[col2], categories=month_order, ordered=True)
            
            cross_tab = pd.crosstab(pnl_temp[col1], pnl_temp[col2])
            
            fig, ax = plt.subplots(figsize=(14, 9))
            
            sns.heatmap(cross_tab, cmap='YlOrRd', annot=True, fmt='d', 
                       linewidths=2, linecolor='white',
                       cbar_kws={'label': 'Trade Count', 'shrink': 0.8},
                       annot_kws={'fontsize': 11, 'fontweight': 'bold'},
                       ax=ax)
            
            ax.set_title(f"Relationship between {col1} and {col2}", 
                        fontsize=16, fontweight='bold', pad=20)
            ax.set_xlabel(col2, fontsize=13, fontweight='bold', labelpad=10)
            ax.set_ylabel(col1, fontsize=13, fontweight='bold', labelpad=10)
            
            ax.tick_params(axis='x', labelsize=11, rotation=45)
            ax.tick_params(axis='y', labelsize=11, rotation=0)
            
            plt.setp(ax.get_xticklabels(), ha='right', rotation_mode='anchor')
            
            plt.tight_layout()
            plt.show()          


--------- These plots...

we can also use the Pandas Profiller to apply the EDA to the User Trade History dataset

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(pnl, title="User Trade History Analysis")
profile.to_file("User Trade History Analysis.html")

# Feature Engineering

Since the realised profit and loss is not calculating the charges, taxes and brokerages paid by the user to the Broker and Exchange so we will be calculating them. 

In [ ]:
# # Lets Calculate the Charges, Taxes and Brokerages for each of the trades.
# # Calculating the Brokerage
# pnl['Brokerage'] = (pnl['Buy value'] + pnl['Sell value']) * 0.0005
# pnl['Brokerage'] = pnl['Brokerage'].apply(lambda x: min(x, 20)) *(1.18)

# # Calculating the Exchange Charges, SEBI Charges and Investor Protection Fund
# pnl['Exchange Charges'] = (pnl['Buy value'] + pnl['Sell value']) * 0.0000297 *(1.18)
# pnl['SEBI Charges'] = (pnl['Buy value'] + pnl['Sell value']) * 0.000001 *(1.18)
# pnl['Investor Protection Fund'] = (pnl['Buy value'] + pnl['Sell value']) * 0.000001 *(1.18)

# # Initialize columns
# pnl['STT'] = 0
# pnl['Stamp Duty'] = 0

# # For Intraday
# pnl.loc[pnl['Remark'] == 'Intraday', 'STT'] = pnl['Sell value'] * 0.00025  *(1.18)
# pnl.loc[pnl['Remark'] == 'Intraday', 'Stamp Duty'] = pnl['Buy value'] * 0.00003 *(1.18)
# pnl.loc[pnl['Remark'] == 'Intraday', 'Demat Charges'] = 0 *(1.18)

# # For Delivery (or others)
# pnl.loc[pnl['Remark'] != 'Intraday', 'STT'] = (pnl['Sell value'] + pnl['Buy value']) * 0.001 *(1.18)
# pnl.loc[pnl['Remark'] != 'Intraday', 'Stamp Duty'] = pnl['Buy value'] * 0.00015 *(1.18)
# pnl.loc[pnl['Remark'] != 'Intraday', 'Demat Charges'] = 13.5 *(1.18)

# # For IPO
# pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'STT'] = (pnl['Sell value']) * 0.001 *(1.18)
# pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'Stamp Duty'] = 0 *(1.18)
# pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'Exchange Charges'] = (pnl['Sell value']) * 0.0000297 *(1.18)
# pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'SEBI Charges'] = (pnl['Sell value']) * 0.000001 *(1.18)
# pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'Investor Protection Fund'] = (pnl['Sell value']) * 0.000001 *(1.18)

# # Calculating the Total Charges including GST
# pnl['Total Charges'] = (pnl['Brokerage'] + pnl['STT'] + pnl['Exchange Charges'] + pnl['SEBI Charges'] + pnl['Investor Protection Fund'] + pnl['Stamp Duty'] + pnl['Demat Charges'])

# # Since we have calculated the charges, taxes and brokerages for each of the trades, we can now drop the unnecessary columns.
# charges_to_drop = ['STT', 'Exchange Charges', 'SEBI Charges', 'Investor Protection Fund', 'Stamp Duty', 'Demat Charges']
# pnl = pnl.drop(columns=charges_to_drop)

# # Adding a new column for Total charges in the PnL dataset
# pnl['Total Charges'] = pnl['Total Charges'].round(4)
# pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)

Now, let's find the Gross and Net pnl and pnl % to better understand the profits and losses. This will give the idea of overall profits user generates. 

In [ ]:
# # Calculating the Gross and Net pnl and pnl %
# pnl['Gross Realised P&L'] = pnl['Sell value'] - pnl['Buy value']
# pnl['Gross Realised PnL %'] = (pnl['Gross Realised P&L'] / pnl['Buy value']) * 100
# pnl['Net Realised P&L'] = pnl['Gross Realised P&L'] - pnl['Total Charges']
# pnl['Net Realised PnL %'] = (pnl['Net Realised P&L'] / pnl['Buy value']) * 100
# pnl = pnl.drop(columns= 'Realised P&L')
# pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)

Now, let's find the holding period to understand the user's preference and find the most profitable number of days for the user.

In [ ]:
# # Convert dates to datetime
# pnl['Buy date'] = pd.to_datetime(pnl['Buy date'], dayfirst=True)
# pnl['Sell date'] = pd.to_datetime(pnl['Sell date'], dayfirst=True)

# # 1. Calculate Holding Period (in days)
# pnl['Holding Period (Days)'] = (pnl['Sell date'] - pnl['Buy date']).dt.days
# pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)